In [1]:
import numpy as np
from synd.core import load_model

Load the SynD model, and the source/target state definitions

In [2]:
model = load_model('coarse-model.synd')
state_definitions = np.load('state_definitions.npz')

In [3]:
source_states = state_definitions['source']
target_states = state_definitions['target']

In [4]:
initial_points = np.concatenate([source_states for x in range(5)])
initial_points

array([0, 0, 0, 0, 0])

Seed the RNG for reproducible outputs

In [5]:
model.rng = np.random.default_rng(seed=50)

Generate a 1000-step trajectory, from our set of 30 source states

In [6]:
sample_trajectories = model.generate_trajectory(
    initial_states = initial_points,
    n_steps = 1000,
)

In [7]:
sample_trajectories

array([[  0,  41,  44, ..., 237, 268, 196],
       [  0,  31,  32, ..., 184, 175, 231],
       [  0,  29,  39, ..., 275, 275, 267],
       [  0,  37,  33, ..., 272, 147, 161],
       [  0,  20,  48, ..., 222, 222, 218]])

Check that the sample trajectories actually reached the target state

In [8]:
np.isin(target_states, sample_trajectories).any()

True

In [9]:
np.save('sample_trajectories.npy', sample_trajectories)